## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [1]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [2]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/airline-safety/airline-safety.csv', 'filename':'airline-safety.csv'},
    { 'url': 'https://raw.githubusercontent.com/fivethirtyeight/data/refs/heads/master/bad-drivers/bad-drivers.csv', 'filename':'bad-drivers.csv'}
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

Overall Download Progress:  50%|█████     | 1/2 [00:00<00:00,  7.34it/s]   

Successfully downloaded: airline-safety.csv



Overall Download Progress: 100%|██████████| 2/2 [00:00<00:00,  7.95it/s]

Successfully downloaded: bad-drivers.csv


### Dataset #1 

Instructions: 
1. Change the header from Dataset #1 to something more descriptive of the dataset
2. Write a few paragraphs about this dataset. Make sure to cover
   1. Describe the important metrics, what units they are in, and giv some sense of what they mean.  For example "Fasting blood glucose in units of mg glucose per deciliter of blood.  Normal values for healthy individuals range from 70 to 100 mg/dL.  Values 100-125 are prediabetic and values >125mg/dL indicate diabetes. Values <70 indicate hypoglycemia. Fasting idicates the patient hasn't eaten in the last 8 hours.  If blood glucose is >250 or <50 at any time (regardless of the time of last meal) the patient's life may be in immediate danger"
   2. If there are any major concerns with the dataset, describe them. For example "Dataset is composed of people who are serious enough about eating healthy that they voluntarily downloaded an app dedicated to tracking their eating patterns. This sample is likely biased because of that self-selection. These people own smartphones and may be healthier and may have more disposable income than the average person.  Those who voluntarily log conscientiously and for long amounts of time are also likely even more interested in health than those who download the app and only log a bit before getting tired of it"
3. Use the cell below to 
    1. load the dataset 
    2. make the dataset tidy or demonstrate that it was already tidy
    3. demonstrate the size of the dataset
    4. find out how much data is missing, where its missing, and if its missing at random or seems to have any systematic relationships in its missingness
    5. find and flag any outliers or suspicious entries
    6. clean the data or demonstrate that it was already clean.  You may choose how to deal with missingness (dropna of fillna... how='any' or 'all') and you should justify your choice in some way
    7. You will load raw data from `data/00-raw/`, you will (optionally) write intermediate stages of your work to `data/01-interim` and you will write the final fully wrangled version of your data to `data/02-processed`
4. Optionally you can also show some summary statistics for variables that you think are important to the project
5. Feel free to add more cells here if that's helpful for you


In [3]:
# This cell web scrapes data on roller coasters from the Roller Coaster Database (RCDB). 
# It defines a list of parks to scrape, fetches the coaster URLs for each park, and then scrapes specific data points for each coaster. 
# The final result is stored in a DataFrame called `df`.

import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from typing import List, Dict

BASE_URL = "https://rcdb.com"
REQUEST_DELAY = 0.01

HEADERS = {
    "User-Agent": "Mozilla/5.0 (educational research)"
}

# This is all of the parks that we will be scraping. The URLs are for the base URLs for their pages on RCDB. 
# This program scraps all of the operating coasters on each park's page.
PARKS = {
    # Cedar Fair (Legacy)
    "Cedar Point": {"url": "https://rcdb.com/4529.htm", "chain": "Cedar Fair"},
    "Kings Island": {"url": "https://rcdb.com/4540.htm", "chain": "Cedar Fair"},
    "Carowinds": {"url": "https://rcdb.com/4542.htm", "chain": "Cedar Fair"},
    "Canada's Wonderland": {"url": "https://rcdb.com/4539.htm", "chain": "Cedar Fair"},
    "Knott's Berry Farm": {"url": "https://rcdb.com/4546.htm", "chain": "Cedar Fair"},
    "California's Great America": {"url": "https://rcdb.com/4541.htm", "chain": "Cedar Fair"},
    "Kings Dominion": {"url": "https://rcdb.com/4544.htm", "chain": "Cedar Fair"},
    "Dorney Park": {"url": "https://rcdb.com/4588.htm", "chain": "Cedar Fair"},
    "Worlds of Fun": {"url": "https://rcdb.com/4533.htm", "chain": "Cedar Fair"},
    "Valleyfair": {"url": "https://rcdb.com/4552.htm", "chain": "Cedar Fair"},
    "Michigan's Adventure": {"url": "https://rcdb.com/4578.htm", "chain": "Cedar Fair"},

    # Six Flags 
    "Six Flags Magic Mountain": {"url": "https://rcdb.com/4532.htm", "chain": "Six Flags"},
    "Six Flags Great Adventure & Safari": {"url": "https://rcdb.com/4534.htm", "chain": "Six Flags"},
    "Six Flags Great America": {"url": "https://rcdb.com/4530.htm", "chain": "Six Flags"},
    "Six Flags Over Texas": {"url": "https://rcdb.com/4531.htm", "chain": "Six Flags"},
    "Six Flags Fiesta Texas": {"url": "https://rcdb.com/4538.htm", "chain": "Six Flags"},
    "Six Flags Over Georgia": {"url": "https://rcdb.com/4535.htm", "chain": "Six Flags"},
    "Six Flags New England": {"url": "https://rcdb.com/4565.htm", "chain": "Six Flags"},
    "Six Flags Discovery Kingdom": {"url": "https://rcdb.com/4711.htm", "chain": "Six Flags"},
    "Six Flags St. Louis": {"url": "https://rcdb.com/4536.htm", "chain": "Six Flags"},
    "Six Flags America": {"url": "https://rcdb.com/4558.htm", "chain": "Six Flags"},
    "Six Flags Darien Lake": {"url": "https://rcdb.com/4581.htm", "chain": "Six Flags"},
    "Six Flags Great Escape": {"url": "https://rcdb.com/4596.htm", "chain": "Six Flags"},
    "La Ronde": {"url": "https://rcdb.com/4567.htm", "chain": "Six Flags"},
    "Frontier City": {"url": "https://rcdb.com/4559.htm", "chain": "Six Flags"},
    "Six Flags Mexico": {"url": "https://rcdb.com/4629.htm", "chain": "Six Flags"},

    # Merlin Parks
    "Legoland California": {"url": "https://rcdb.com/4733.htm", "chain": "Legoland"},
    "Legoland Florida": {"url": "https://rcdb.com/5589.htm", "chain": "Legoland"},
    "Legoland New York": {"url": "https://rcdb.com/14429.htm", "chain": "Legoland"},
    "Legoland Billund": {"url": "https://rcdb.com/4903.htm", "chain": "Legoland"},
    "Legoland Deutschland" : {"url": "https://rcdb.com/5059.htm", "chain": "Legoland"},
    "Legoland Dubai": {"url": "https://rcdb.com/13462.htm", "chain": "Legoland"},
    "Legoland Windsor": {"url": "https://rcdb.com/13644.htm", "chain": "Legoland"},
    "Legoland Malaysia": {"url": "https://rcdb.com/9575.htm", "chain": "Legoland"},
    "Legoland Korea": {"url": "https://rcdb.com/12557.htm", "chain": "Legoland"},
    "Legoland Shanghai": {"url": "https://rcdb.com/19273.htm", "chain": "Legoland"},
    "Alton Towers": {"url": "https://rcdb.com/4796.htm", "chain": "Merlin"},
    "Chessington World of Adventures": {"url": "https://rcdb.com/4798.htm", "chain": "Merlin"},
    "Heide Park Resort": {"url": "https://rcdb.com/4874.htm", "chain": "Merlin"},
    "Gardaland": {"url": "https://rcdb.com/4866.htm", "chain": "Merlin"},
    "Thorpe Park": {"url": "https://rcdb.com/4814.htm", "chain": "Merlin"},

    # Disney
    "Magic Kingdom": {"url": "https://rcdb.com/4597.htm", "chain": "Disney"},
    "EPCOT": {"url": "https://rcdb.com/15503.htm", "chain": "Disney"},
    "Hollywood Studios": {"url": "https://rcdb.com/4735.htm", "chain": "Disney"},
    "Animal Kingdom": {"url": "https://rcdb.com/5109.htm", "chain": "Disney"},
    "Disneyland": {"url": "https://rcdb.com/4571.htm", "chain": "Disney"},
    "Disney California Adventure": {"url": "https://rcdb.com/4783.htm", "chain": "Disney"},
    "Disneyland Paris": {"url": "https://rcdb.com/4864.htm", "chain": "Disney"},
    "Disneyland Paris - Walt Disney Studios Park": {"url": "https://rcdb.com/5054.htm", "chain": "Disney"},
    "Tokyo Disneyland": {"url": "https://rcdb.com/4959.htm", "chain": "Disney"},
    "Tokyo DisneySea": {"url": "https://rcdb.com/5073.htm", "chain": "Disney"},
    "Shanghai Disneyland": {"url": "https://rcdb.com/6556.htm", "chain": "Disney"},
    "Hong Kong Disneyland": {"url": "https://rcdb.com/5279.htm", "chain": "Disney"},

    # Universal 
    "Universal Studios Florida": {"url": "https://rcdb.com/4736.htm", "chain": "Universal"},
    "Islands of Adventure": {"url": "https://rcdb.com/4734.htm", "chain": "Universal"},
    "Epic Universe": {"url": "https://rcdb.com/17569.htm", "chain": "Universal"},
    "Universal Studios Hollywood": {"url": "https://rcdb.com/5265.htm", "chain": "Universal"},
    "Universal Studios Singapore": {"url": "https://rcdb.com/4859.htm", "chain": "Universal"},
    "Universal Studios Japan": {"url": "https://rcdb.com/5492.htm", "chain": "Universal"},
    "Universal Studios Beijing": {"url": "https://rcdb.com/17287.htm", "chain": "Universal"},

    # United Parks
    "Busch Gardens Tampa": {"url": "https://rcdb.com/4543.htm", "chain": "United Parks"},
    "Busch Gardens Williamsburg": {"url": "https://rcdb.com/4548.htm", "chain": "United Parks"},
    "SeaWorld Orlando": {"url": "https://rcdb.com/4746.htm", "chain": "United Parks"},
    "SeaWorld San Antonio": {"url": "https://rcdb.com/4601.htm", "chain": "United Parks"},
    "SeaWorld San Diego": {"url": "https://rcdb.com/5320.htm", "chain": "United Parks"},

    # Herschend Family Entertainment
    "Adventureland": {"url": "https://rcdb.com/4576.htm", "chain": "Herschend"},
    "Dollywood": {"url": "https://rcdb.com/4593.htm", "chain": "Herschend"},
    "Kennywood": {"url": "https://rcdb.com/4553.htm", "chain": "Herschend"},
    "Kentucky Kingdom": {"url": "https://rcdb.com/4563.htm", "chain": "Herschend"},
    "Lake Compounce": {"url": "https://rcdb.com/4683.htm", "chain": "Herschend"},
    "Silver Dollar City": {"url": "https://rcdb.com/4579.htm", "chain": "Herschend"},
    "Wild Adventures": {"url": "https://rcdb.com/4703.htm", "chain": "Herschend"},

    # Parques Reunidos
    "Bobbejaanland": {"url": "https://rcdb.com/4846.htm", "chain": "Parques Reunidos"},
    "MoviePark Germany": {"url": "https://rcdb.com/4869.htm", "chain": "Parques Reunidos"},
    "Mirabilandia": {"url": "https://rcdb.com/4793.htm", "chain": "Parques Reunidos"},
    "Parque de Atracciones": {"url": "https://rcdb.com/4928.htm", "chain": "Parques Reunidos"},
    "Parque Warner Madrid": {"url": "https://rcdb.com/5028.htm", "chain": "Parques Reunidos"},
    "TusenFryd": {"url": "https://rcdb.com/4901.htm", "chain": "Parques Reunidos"},

    # Other 
    "Efteling": {"url": "https://rcdb.com/4839.htm", "chain": "Other"},
    "Nagashima Spa Land": {"url":"https://rcdb.com/4958.htm", "chain": "Other"},
    "Europa Park": {"url": "https://rcdb.com/4870.htm", "chain": "Other"},
    "Port Adventura": {"url": "https://rcdb.com/4792.htm", "chain": "Other"},
    "Phantasialand": {"url": "https://rcdb.com/4872.htm", "chain": "Other"},
    "Nickelodeon Universe": {"url": "https://rcdb.com/15593.htm", "chain": "Other"},
    "Fuji-Q Highland": {"url": "https://rcdb.com/4961.htm", "chain": "Other"},
    "Energylandia": {"url": "https://rcdb.com/12068.htm", "chain": "Other"},
    "Liseberg":{ "url": "https://rcdb.com/4909.htm", "chain": "Other"},
    "Hersheypark": {"url": "https://rcdb.com/4545.htm", "chain": "Other"},
    "Holiday World": {"url": "https://rcdb.com/4554.htm", "chain": "Other"},
    "Silverwood": {"url": "https://rcdb.com/4564.htm", "chain": "Other"},
    "Lagoon":{ "url": "https://rcdb.com/4570.htm", "chain": "Other"},
    "Knoebels": {"url": "https://rcdb.com/4557.htm", "chain": "Other"},
    "Mt. Olympus Water & Theme Park": {"url": "https://rcdb.com/4561.htm", "chain": "Other"},
    "Pleasure Beach Blackpool": {"url": "https://rcdb.com/4795.htm", "chain": "Other"},
}

# This function takes a URL, makes a GET request to it, and returns a BeautifulSoup object if the request is successful. 
# If there's an error during the request, it prints an error message and returns None.
def get_soup(url: str) -> BeautifulSoup | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        r.raise_for_status()
        return BeautifulSoup(r.text, "html.parser")
    except Exception as e:
        print(f"Fetch failed: {url} ({e})")
        return None

# This function takes a park URL, fetches the page, and looks for the table that lists the operating coasters. 
# It then extracts the URLs for each coaster and returns them as a list.
def get_coaster_urls(park_url: str) -> List[str]:
    soup = get_soup(park_url)
    if not soup:
        return []

    coaster_urls = []

    for table in soup.find_all("table"):
        headers = table.find_all("th")
        if not headers:
            continue

        header_text = "".join(h.get_text(strip=True) for h in headers)

        # RCDB coaster tables always contain these substrings
        if "Name" not in header_text or "Opened" not in header_text:
            continue

        # This is the operating coaster table
        for row in table.find_all("tr"):
            cells = row.find_all("td")
            if len(cells) < 2:
                continue

            link = cells[1].find("a", href=True)
            if link and re.fullmatch(r"/\d+\.htm", link["href"]):
                coaster_urls.append(BASE_URL + link["href"])

        break  # only first valid coaster table

    return list(set(coaster_urls))

# This scrapes the data for a single coaster given its URL. It looks for specific patterns in the text to extract the relevant information.
def scrape_coaster_data(url: str, park: str, chain: str) -> Dict | None:
    soup = get_soup(url)
    if not soup:
        return None

    name = soup.find("h1").get_text(strip=True)

    text = soup.get_text(" ", strip=True)

    def grab(pattern):
        m = re.search(pattern, text)
        return m.group(1) if m else None

    return {
        "Chain": chain,
        "Park": park,
        "Name": name,
        "Height": grab(r"Height[:\s]*(\d+\.?\d*\s*(?:ft|m))"),
        "Speed": grab(r"Speed[:\s]*(\d+\.?\d*\s*(?:mph|km/h))"),
        "Length": grab(r"Length[:\s]*(\d+\.?\d*\s*(?:ft|m))"),
        "Inversions": grab(r"Inversions?[:\s]*(\d+)"),
        "URL": url,
    }

# This function iterates through all the parks, scrapes the coaster URLs for each park, and then scrapes the data for each coaster. 
# Compiles all the data into a list of dictionaries, which is then converted into a DataFrame at the end.
def scrape_all_parks() -> pd.DataFrame:
    rows = []

    for i, (park, info) in enumerate(PARKS.items(), 1):
        print(f"\n[{i}/{len(PARKS)}] {park}")

        urls = get_coaster_urls(info["url"])
        print(f"  Found {len(urls)} coasters")

        for j, url in enumerate(urls, 1):
            data = scrape_coaster_data(url, park, info["chain"])
            if data:
                print(f"    [{j}] {data['Name']}")
                rows.append(data)
            time.sleep(REQUEST_DELAY)

    return pd.DataFrame(rows)

global df
df = scrape_all_parks() # Calls the function to scrape all the parks and returns to df
print(f"\n Done. {len(df)} coasters scraped.")


[1/94] Cedar Point
  Found 18 coasters
    [1] Wilderness Run
    [2] Siren’s Curse
    [3] Woodstock Express
    [4] Magnum XL-200
    [5] Maverick
    [6] Cedar Creek Mine Ride
    [7] Raptor
    [8] Wild Mouse
    [9] Valravn
    [10] GateKeeper
    [11] Iron Dragon
    [12] Blue Streak
    [13] Rougarou
    [14] Top Thrill 2
    [15] Steel Vengeance
    [16] Millennium Force
    [17] Gemini
    [18] Corkscrew

[2/94] Kings Island
  Found 15 coasters
    [1] Flight of Fear
    [2] Racer
    [3] Invertigo
    [4] Woodstock Express
    [5] Mystic Timbers
    [6] Diamondback
    [7] Beast
    [8] Banshee
    [9] Woodstock’s Air Rail
    [10] Queen City Stunt Coaster
    [11] Great Pumpkin Coaster
    [12] Bat
    [13] Snoopy's Soap Box Racers
    [14] Orion
    [15] Adventure Express

[3/94] Carowinds
  Found 14 coasters
    [1] Ricochet
    [2] Fury 325
    [3] Vortex
    [4] Copperhead Strike
    [5] Hurler
    [6] Carolina Goldrusher
    [7] Thunder Striker
    [8] Carolina Cyclone

In [18]:
nulls = df[df.isnull().any(axis=1)]

#replacing nulls with values advertised on the theme parks themselves websit
# from https://www.sixflags.com/kingsisland/attractions/the-great-pumpkin-coaster
df.loc[(df['Name'] == 'Great Pumpkin Coaster') & (df['Park'] == "Kings Island"), 'Speed'] = '8 mph'

# from https://www.sixflags.com/carowinds/attractions/carolina-goldrusher
df.loc[(df['Name'] == 'Carolina Goldrusher') & (df['Park'] == "Carowinds"), 'Speed'] = '30 mph'
df.loc[(df['Name'] == 'Carolina Goldrusher') & (df['Park'] == "Carowinds"), 'Height'] = '43 ft'

# from https://www.sixflags.com/kingsisland/attractions/woodstock-express
df.loc[(df['Name'] == 'Woodstock Express') & (df['Park'] == "Kings Island"), 'Speed'] = '35 mph'
df.loc[(df['Name'] == 'Woodstock Express') & (df['Park'] == "Kings Island"), 'Height'] = '39 ft'

# from https://www.sixflags.com/knotts/attractions/coast-rider
df.loc[(df['Name'] == 'Coast Rider') & (df['Park'] == "Knott's Berry Farm"), 'Speed'] = '37 mph'


# from https://www.sixflags.com/kingsdominion/attractions/grizzly
df.loc[(df['Name'] == 'Grizzly') & (df['Park'] == 'Kings Dominion'), 'Speed'] = '51 mph'


# from https://www.sixflags.com/dorneypark/attractions/woodstock-express
df.loc[(df['Name'] == 'Woodstock Express') & (df['Park'] == 'Dorney Park'), 'Speed'] = '10 mph'

# from https://www.sixflags.com/worldsoffun/attractions/cosmic-coaster
df.loc[(df['Name'] == 'Cosmic Coaster') & (df['Park'] == 'Worlds of Fun'), 'Speed'] = '5 mph'
df.loc[(df['Name'] == 'Cosmic Coaster') & (df['Park'] == 'Valleyfair'), 'Speed'] = '15 ft'

# from https://www.sixflags.com/valleyfair/attractions/cosmic-coaster
df.loc[(df['Name'] == 'Cosmic Coaster') & (df['Park'] == 'Valleyfair'), 'Speed'] = '20 mph'
df.loc[(df['Name'] == 'Cosmic Coaster') & (df['Park'] == 'Valleyfair'), 'Speed'] = '14 ft'


# from https://www.sixflags.com/magicmountain/attractions/canyon-blaster
df.loc[(df['Name'] == 'Canyon Blaster') & (df['Park'] == 'Six Flags Magic Mountain'), 'Speed'] = '10 mph'

# from https://finance.yahoo.com/news/six-flags-great-adventure-launch-070000570.html
df.loc[(df['Name'] == 'Flash: Vertical Velocity') & (df['Park'] == 'Six Flags Great Adventure'), 'Height'] = '172 ft'

# from https://www.sixflags.com/greatamerica/attractions/little-dipper
df.loc[(df['Name'] == 'Little Dipper') & (df['Park'] == 'Six Flags Great America'), 'Speed'] = '20 mph'

# from https://www.sixflags.com/overtexas/attractions/runaway-mine-train
df.loc[(df['Name'] == 'Runaway Mine Train') & (df['Park'] == 'Six Flags Over Texas'), 'Speed'] = '35 mph'

# from https://www.sixflags.com/overtexas/attractions/wile-e-coyotes-grand-canyon-blaster
df.loc[(df['Name'] == "Wile E. Coyote's Grand Canyon Blaster") & (df['Park'] == 'Six Flags Over Texas'), 'Speed'] = '15 mph'
df.loc[(df['Name'] == "Wile E. Coyote's Grand Canyon Blaster") & (df['Park'] == 'Six Flags Over Texas'), 'Speed'] = '15 mph'

# from https://www.sixflags.com/overtexas/attractions/mini-mine-train
df.loc[(df['Name'] == 'Mini Mine Train') & (df['Park'] == 'Six Flags Over Texas'), 'Speed'] = '20 mph'

# from https://www.sixflags.com/fiestatexas/attractions/road-runner-express
df.loc[(df['Name'] == 'Roadrunner Express') & (df['Park'] == 'Six Flags Fiesta Texas'), 'Speed'] = '35 mph'

# from https://www.sixflags.com/newengland/attractions/pandemonium
df.loc[(df['Name'] == 'Pandemonium') & (df['Park'] == 'Six Flags Over Georgia'), 'Speed'] = '31 mph'
df.loc[(df['Name'] == 'Pandemonium') & (df['Park'] == 'Six Flags Over Georgia'), 'Length'] = '1351 mph'

# from https://coasterpedia.net/wiki/Dragon_(Legoland_California)
df.loc[(df['Name'] == 'Dragon') & (df['Park'] == 'Legoland California'), 'Speed'] = '28.5 mph'
df.loc[(df['Name'] == 'Dragon') & (df['Park'] == 'Legoland California'), 'Height'] = '42.7 meters'

# from https://www.altontowers.com/explore/theme-park/rides-attractions/runaway-mine-train/
df.loc[(df['Name'] == 'Runaway Mine Train') & (df['Park'] == 'Alton Towers'), 'Height'] = '11 meters'

# from https://www.altontowers.com/explore/theme-park/rides-attractions/the-smiler/
df.loc[(df['Name'] == 'Smiler') & (df['Park'] == 'Alton Towers'), 'Height'] = '30 meters'

# from https://www.altontowers.com/explore/theme-park/rides-attractions/spinball-whizzer/
df.loc[(df['Name'] == 'Spinball Whizzer') & (df['Park'] == 'Alton Towers'), 'Speed'] = '60 kph'

# from https://www.altontowers.com/explore/theme-park/rides-attractions/wicker-man/
df.loc[(df['Name'] == 'Wicker Man') & (df['Park'] == 'Alton Towers'), 'Speed'] = '20 meters'

# from https://coasterpedia.net/wiki/Vampire_(Chessington_World_of_Adventures)
df.loc[(df['Name'] == 'Vampire') & (df['Park'] == 'Chessington World of Adventure'), 'Speed'] = '21 meters'

# from https://www.thorpepark.com/explore/theme-park/rides/saw-the-ride/
df.loc[(df['Name'] == 'Saw - The Ride') & (df['Park'] == 'Thorpe Park'), 'Speed'] = '55 mph'

# from https://www.universalorlando.com/web/en/us/things-to-do/rides-attractions/stardust-racers
df.loc[(df['Name'] == 'Stardust Racers') & (df['Park'] == 'Epic Universe'), 'Length'] = '5000 ft'


nulls.to_csv('output.csv')

In [12]:
def standardize_len(str_in):
    try:
        str_in = str_in.lower()
        str_in = str_in.strip()
        if 'ft' in str_in:
            str_in = str_in.replace('ft', '')
            output = float(str_in)
        elif 'meters' in str_in or 'm' in str_in:
            str_in = str_in.replace('meters', '')
            str_in = str_in.replace('m', '')
            output = float(str_in)
            output = 3.28084 * output
        else:
            output = np.nan
    except:
        output = np.nan
    return output

def standardize_speed(str_in):
    try:
        str_in = str_in.lower()
        str_in = str_in.strip()
        
        if 'mph' in str_in:
            str_in = str_in.replace('mph', '')
            output = float(str_in)
        elif 'kph' in str_in:
            str_in = str_in.replace('kph', '')
            output = float(str_in)
            output = 0.621371 * output
        else:
            output = np.nan
    except:
        output = np.nan
    return output


In [15]:
#drop null values
cleaned_df = df.dropna(how = 'any')

cleaned_df.loc[:, 'Height'] = cleaned_df['Height'].apply(standardize_len)
cleaned_df.loc[:, 'Length'] = cleaned_df['Length'].apply(standardize_len)
cleaned_df.loc[:, 'Speed'] = cleaned_df['Speed'].apply(standardize_speed)

,Chain,Park,Name,Height,Speed,Length,Inversions,URL
0,Cedar Fair,Cedar Point,Wilderness Run,19.0,6.0,443.0,0,https://rcdb.com/15.htm
1,Cedar Fair,Cedar Point,Siren’s Curse,160.0,58.4,2965.9,2,https://rcdb.com/22124.htm
2,Cedar Fair,Cedar Point,Woodstock Express,42.7,28.5,1099.1,0,https://rcdb.com/553.htm
3,Cedar Fair,Cedar Point,Magnum XL-200,205.0,72.0,5106.0,0,https://rcdb.com/11.htm
4,Cedar Fair,Cedar Point,Maverick,105.0,70.0,4450.0,2,https://rcdb.com/3570.htm
...,...,...,...,...,...,...,...,...
675,Other,Pleasure Beach Blackpool,Grand National,62.0,40.0,3302.0,0,https://rcdb.com/792.htm
676,Other,Pleasure Beach Blackpool,Nickelodeon Streak,61.0,35.0,2293.0,0,https://rcdb.com/793.htm
677,Other,Pleasure Beach Blackpool,Big One,213.0,74.0,5497.0,0,https://rcdb.com/775.htm
682,Other,Pleasure Beach Blackpool,Infusion,109.3,49.7,2260.5,5,https://rcdb.com/3672.htm


### Dataset #2 

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE

## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them